<a href="https://colab.research.google.com/github/JoshJingtianWang/TCGA-Splicing-Data-Cleaning-and-Filtering/blob/main/getcandidates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get threshold from ORAI2

Simply substract the medians of Normal PSIs and K700E PSIs






In [ ]:
srun -c 4 --pty --x11 /bin/bash -i #enable x forwarding

import pandas as pd
import numpy as np

#read in PSI values
PSI = pd.read_csv('merged_PSI_for_R.csv', sep='\t')
PSI.loc[PSI['Type']=='Normal','PSI']
#turn normal PSI into array (ORAI2 only)
normalarray=PSI.loc[PSI['Type']=='Normal','PSI'].values
#turn K700E PSI into array (ORAI2 only)
K700Earray=PSI.loc[PSI['Type']=='K700E','PSI'].values

#get median
np.median(normalarray) #1.0
np.median(K700Earray) #0.3666665
#1.0-0.3666665=0.6333335

# First Filter: Test for similarity between normal PSIs and 1 (or 0). Filter out genes where normal are not similar enough to 1 (or 0)
# Second filter: substract the medians of Normal PSIs and geneA PSIs. Filter out ones that do not meet ORAI2 threshold
# Third filter: PSI of gene A of patient x needs to be significantly different from that of normal samples.

In [ ]:
#the wilcoxon p of the normal ORAI2 PSI came out as sig dif from 1.0. Cannot use wilcoxon
#for now, use "median of normal PSIs must == 1.0" as filter

#read in normal table. filter out rows with fewer than 20 non-NaN datapoints. get median for each row. filter out rows where median is less than 1.0.


import pandas as pd
import datetime as dt
import re
from scipy.stats import wilcoxon

#read in normal table
start = dt.datetime.now()
print('{} seconds: Starting...'.format((dt.datetime.now() - start).seconds))
normdf = pd.read_csv('normaltablewID.txt', encoding='utf-8', sep='\t', header=0)
print('{} seconds: completed'.format((dt.datetime.now() - start).seconds)) #takes about 17 seconds to complete on interactive node (4 cores)

#keep rows with at least 20 non-'nan' datapoints
#https://stackoverflow.com/questions/23203638/filter-out-rows-with-more-than-certain-number-of-nan
normdf_nafiltered=normdf.dropna(thresh=20) #filtering out rows with more than 20 NaNs
normdf_nafiltered.head
normdf_nafiltered.shape

#get mean of each row
#https://www.delftstack.com/api/python-pandas/pandas-dataframe-dataframe.mean-function/
normdf_nafiltered['median'] = normdf_nafiltered.median(numeric_only=True, axis=1,skipna=True) #median as a new column

#remove rows where 'median' != 0 or 1
normfiltered1=normdf_nafiltered.loc[(normdf_nafiltered['median']==1.0) | (normdf_nafiltered['median']==0.0)]



In [ ]:
#this writes filtering results to a nested dictionary


#get headers
patientheaders=pd.read_csv('headers',sep='\t')
patientheader=list(patientheaders)
linelength=len(patientheader)
finalresult={}
patientcount=len(patientheader)-6
ORAI2cutoff=0.6333335

#start filtering
with open("threeprimePSItrimmedheader") as fp:
        for line in fp:
                line=line.split('\t') #turn line into list
                line=[s.strip() for s in line] ##remove potential newline character https://stackoverflow.com/questions/12533955/string-split-on-new-line-tab-and-some-number-of-spaces
                eventID=line[0] #take first item as event id
                #print('eventid is ' + eventID)
                IDinfo=line[0:6] #this is the ID info of each line
                PSIline=line[6:] #this is all the PSIs of each line
                IDheader=['event_id','event_type','event_chr','event_coordinates','alt_region_coordinates','gene_name']
                finalline=dict(zip(IDheader, IDinfo)) #preparing finalline (a dictionary)
                #print(finalline)

                if eventID not in normfiltered1['event_id'].tolist(): #pandas data types are strange. safer to convert them to lists or arrays
                        print(eventID + 'is NOT in normal table')
                        continue #if event ID not found, if means normals don't meet filter (too many NaNs or median not 1 or 0)
                else:
                        normalPSImedian=normfiltered1.loc[normfiltered1['event_id']==eventID, 'median']
                        print('event id is in normal table, normal median is ' + str(normalPSImedian))
                        count=0 #counter for generating patient number
                        for PSI in PSIline:
                                count+=1
                                patientnumber='patient'+str(count) #patient number for dictionary
                                print("working on " + eventID + ' of ' + patientnumber)
                                if not re.match(r'^\d\.\d+', PSI): #check if PSI match the 'digit dot digit digit...' format, if not then it's skipped
                                        continue
                                else:
                                        PSIdiff = float(PSI) - float(normalPSImedian)
                                        #must use float() for both https://stackoverflow.com/questions/36921951/truth-value-of-a-series-is-ambiguous-use-a-empty-a-bool-a-item-a-any-o
                                        if (PSIdiff < ORAI2cutoff) and (PSIdiff > -ORAI2cutoff):
                                                print('did not meet ORAI2 cutoff')
                                        else:
                                                normalarray=normfiltered1.loc[normfiltered1['event_id']==eventID].iloc[:,6:].dropna(axis=1).values.ravel() #array of normal PSIs. Flatten to 1 dimension with ravel()
                                                #https://stackoverflow.com/questions/9057379/correct-and-efficient-way-to-flatten-array-in-numpy-in-python
                                                w, p = wilcoxon(normalarray-PSI,zero_method='wilcox')
                                                finalline[patientnumber]=p #need to
                                                print(str(p)+' met ORAI2 cutoff')

                finalresult[eventID] = finalline

print('length of dictionary is ' + str(len(finalresult)))

print('{} seconds: completed'.format((dt.datetime.now() - start).seconds))

#save dictionary to file https://stackoverflow.com/questions/19201290/how-to-save-a-dictionary-to-a-file/32216025
import pickle
with open('candidatepvalues.pkl', 'wb') as f:
    pickle.dump(finalresult, pickle.HIGHEST_PROTOCOL)

#Processing nested dictionary

1. get patient-count table (for each patient, how many events do they have)

In [ ]:
#process the nested dictionary
#mydict={event1:{geneinfo:xxx, patient1:p, patient2:p...}, event2:{geneinfo:xxx, patient1:p, patient2:p...}...}

#load dictionary from file
import pickle
with open('candidatepvalues.pkl', 'rb') as f:
  mydict=pickle.load(f)

#getting test count for multiple testing correction
testcount=0
for event in mydict:
  testcount=testcount+len(mydict[event])-6
#Out: 1362530

#multiple testing correction
import re
for event in mydict:  
  for patient in list(mydict[event]): #https://stackoverflow.com/questions/11941817/how-to-avoid-runtimeerror-dictionary-changed-size-during-iteration-error
    pvalue=mydict[event][patient]
    if re.match(r'^patient\d+', patient):
    #if patient.startswith('patient'):
      pvalueadj=pvalue*testcount
      if pvalueadj >= 0.001:
        mydict[event].pop(patient)

#getting test count AFTER multiple testing correction
testcount2=0
for event in mydict:
  testcount2=testcount2+len(mydict[event])-6
print(testcount2)
#Out: 950823


#getting list of patients (including duplicates)  who passed the filters
patientlist=[]
for event in mydict:
  for patient in mydict[event]:
    if re.match(r'^patient\d+', patient):
    #if patient.startswith('patient'):
      patientlist.append(patient)
print(len(patientlist))

#list of unique patients who passed the filters
patientlistunique=list(set(patientlist))
print(len(patientlistunique))
#Out 10019

#get total unfiltered patient count
with open('headers') as f:
    patientcount = f.readline().strip() #this reads only first line https://stackoverflow.com/questions/1904394/read-only-the-first-line-of-a-file
patientcount=patientcount.split('\t') #turn line into list
patientcount=patientcount[6:] #remove gene info at the start
len(patientcount)

#dictionary of {patient:count of events that passed filters}
patientdict={}
for i in patientlist:
  patientdict[i] = patientdict.get(i, 0) + 1 #https://stackoverflow.com/questions/3496518/using-a-dictionary-to-count-the-items-in-a-list/6582852#6582852

#turn patient dictionary into a dataframe for further processing in R https://stackoverflow.com/questions/18837262/convert-python-dict-into-a-dataframe
patientdf=pd.DataFrame(patientdict.items(), columns=['Patient', 'Count'])
with open('patientdf_for_R.csv', 'w') as file:
     patientdf.to_csv(file, sep='\t')

2. get event-count table (for each event, how many patients are inflicted)

In [ ]:
#load dictionary from file
import pickle
with open('candidatepvalues.pkl', 'rb') as f:
  mydict=pickle.load(f)

eventcount={}
for event in mydict:
  eventcount[event]=len(mydict[event])-6

import pandas as pd
eventcount=pd.DataFrame(eventcount.items(), columns=['Event', 'Count'])
with open('eventdf_for_R.csv', 'w') as file:
     eventcount.to_csv(file, sep='\t')

# Generating figures in R

In [ ]:
#make histogram from the patient count 
srun -c 4 --pty --x11 /bin/bash -i
#R
module load R
R

library(ggplot2)
patientcount <- read.table("patientdf_for_R.csv",header=TRUE,sep='\t')
head(patientcount)
patientcountsorted<-patientcount[order(patientcount$Count),] #sorting by Count

# Basic histogram http://www.sthda.com/english/wiki/ggplot2-histogram-plot-quick-start-guide-r-software-and-data-visualization
ggplot(patientcountsorted, aes(x=Count)) + geom_histogram()
# Change the width of bins
ggplot(patientcountsorted, aes(x=Count)) + 
  geom_histogram(binwidth=1)

# Change colors
png('patientcount.png', width = 1200, height = 1000, res = 200) #res option affects font size

ggplot(patientcountsorted, aes(x=Count)) + 
  geom_histogram(fill="darkred", binwidth=1) +
  xlab("Eligible event counts") + ylab("Patient counts") +
  theme_minimal()

dev.off()




In [ ]:
#make histogram from the event count 
srun -c 4 --pty --x11 /bin/bash -i
#R
module load R
R

library(ggplot2)
eventcount <- read.table("eventdf_for_R.csv",header=TRUE,sep='\t')
head(eventcount)
nrow(eventcount) #Out: 101608

eventcount$X=NULL
#remove events with zero count, since most events have zero count, it greatly skews the plot making it unreadable
eventcountnozero<-eventcount[eventcount$Count>0,] 
nrow(eventcountnozero) #Out: 19298

eventcountT500<-head(eventcount[order(eventcount$Count,decreasing=TRUE),],500) #sorting by Count, top 500 events


# Basic histogram http://www.sthda.com/english/wiki/ggplot2-histogram-plot-quick-start-guide-r-software-and-data-visualization
ggplot(eventcountnozero, aes(x=Count)) + geom_histogram()
# Change the width of bins
ggplot(eventcountnozero, aes(x=Count)) + 
  geom_histogram(binwidth=1)
# Change colors
p<-ggplot(eventcountnozero, aes(x=Count)) + 
  geom_histogram(color="black", fill="white")
p


#Barplot
png('eventcountT100.png', width = 1200, height = 1000, res = 200) #res option affects font size

ggplot(data=eventcountT100, aes(x=reorder(Event,-Count), y=Count)) +
  geom_bar(stat = "identity", fill="steelblue") +
  xlab("Top 100 Events") + ylab("Number of Patients") +
  theme_minimal() +
  scale_y_continuous(breaks=seq(0,5000,1000)) +
  scale_color_brewer(palette="Dark2") +
  theme(legend.position='none') +
  theme(axis.text.x=element_blank(),
        axis.ticks.x=element_blank())

dev.off()

#Make graphs comparing PSIs of cancer v normal for Top events (including ORAI2)

In [ ]:
#PRL10, P4HB, CTSB, ORAI2
